In [1]:
#  INSTALACIÓN Y CARGA DE DEPENDENCIAS
!pip install transformers sentence-transformers accelerate numpy pandas pyarrow datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.7 MB/s eta 0:00:00:00:0100:01
  Attempting u

In [2]:
!pip install faiss-cpu bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 8.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.0 MB/s eta 0:00:00:00:0100:01


Es necesario generar un token de hugginface y darle permisos de lectura para poder acceder al repositorio de mistral.<br>
Una vez creado el token hay q añadirlo a secrets de kagle bajo el nombre de HF_TOKEN

In [3]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
if hf_token:
    login(token=hf_token, add_to_git_credential=False)
    print("Login en Hugging Face exitoso.")
else:
    raise "Error token no definido"

Login en Hugging Face exitoso.


In [4]:
# ==============================================================================
#  DEPENDENCIAS Y LIBRERÍAS
# ==============================================================================
import pandas as pd
import numpy as np
import torch
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import warnings
warnings.filterwarnings("ignore") # Ignorar warnings de tokenizers y modelos


# ==============================================================================
# CARGA Y PROCESAMIENTO DEL CORPUS
# ==============================================================================
def cargar_corpus():
    print("--- 1. Cargando Dataset y Corpus ---")
    ds = load_dataset("dhruvvaidh/cover-letter-dataset-llama3")
    train_df = ds["train"].to_pandas()

    def canonize_row(r):
        output = str(r.get("Output", "")).strip()
        return {
            "doc_id": r.name,
            "text_for_rag": output,
        }

    corpus_df = train_df.apply(canonize_row, axis=1, result_type="expand")
    corpus_list = corpus_df.to_dict('records')
    textos_para_indexar = corpus_df['text_for_rag'].tolist()
    
    print(f"Corpus cargado: {len(corpus_list)} documentos.")
    return corpus_list, textos_para_indexar

corpus_list, textos_para_indexar = cargar_corpus()

2025-11-24 12:03:42.538449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763985822.703366      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763985822.753483      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

data/train-00000-of-00001.parquet:   0%|          | 0.00/841k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/367k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/349 [00:00<?, ? examples/s]

Corpus cargado: 813 documentos.


In [13]:
# ==============================================================================
# INDEXACIÓN (modelo BGE + FAISS vectorial) 
# ==============================================================================
print("\n--- 2. Generando Embeddings e Índice FAISS (BGE) ---")
EMBED_MODEL_ID = "BAAI/bge-large-en-v1.5"

embedder = SentenceTransformer(EMBED_MODEL_ID)
doc_embeddings = embedder.encode(
    textos_para_indexar, 
    batch_size=32, 
    show_progress_bar=True, 
    normalize_embeddings=True
)
doc_embeddings = np.array(doc_embeddings, dtype="float32")
d_dimension = doc_embeddings.shape[1]

index = faiss.IndexFlatIP(d_dimension)
index.add(doc_embeddings)
print(f"Índice FAISS creado en CPU con {index.ntotal} vectores.")


# ==============================================================================
# FUNCIÓN DE BÚSQUEDA 
# ==============================================================================
def buscar_candidatos(query, k=3):
    if index is None or embedder is None:
        return []
    
    q_text = "Represent this sentence for searching relevant passages: " + query
    q_emb = embedder.encode([q_text], normalize_embeddings=True)
    q_emb = np.array(q_emb, dtype="float32")
    
    scores, indices = index.search(q_emb, k)
    
    resultados = []
    for idx, score in zip(indices[0], scores[0]):
        if idx != -1:
            doc_data = corpus_list[idx]
            resultados.append({
                "id": doc_data['doc_id'],
                "score": float(score),
                "context": doc_data['text_for_rag'], # Cover Letter completa
            })
    return resultados

# ==============================================================================
#  FUNCIÓN DE RANKING
# ==============================================================================
def generar_ranking_llm(ranker_pipeline, job_offer, candidatos):
    """Genera un ranking usando tokenizer y una plantilla de chat."""
    
    # Construir el texto de los candidatos
    contexto_str = ""
    for i, c in enumerate(candidatos, 1):
        perfil = c['context'].replace("\n", " ").strip()
        contexto_str += f"CANDIDATE {i} (ID {c['id']}) - COVER LETTER: {perfil}\n\n"
    
    
   # Definición de mensajes con Prompt Engineering avanzado
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert Technical Recruiter. Your job is to evaluate candidates based strictly "
                "on how well their application matches the provided Job Description. "
                "You must provide critical, comparative reasoning for your final choice in 3-5 lines."
            )
        },
        {
            "role": "user",
            "content": f"""
    ### JOB DESCRIPTION:
    {job_offer}
        
    ### CANDIDATES TO EVALUATE:
    {contexto_str}
        
    ### YOUR TASK:
    Analyze all candidates and determine the SINGLE candidate who is the **Best Fit** for the job based on the provided cover letters.
    
    ### REQUIRED OUTPUT FORMAT:
    You must provide the output using this simple, structured format. Do NOT include any numbering, introductory phrases, or text before or after this format.
    
    BEST CANDIDATE ID: [The ID of the single best candidate]
    OVERALL MATCH: [Best/Good/Weak]
    REASONING: [Explain specifically WHY this candidate is the best fit overall, comparing their key skills against the job and the other candidates. The reasoning must be 4-6 lines.]
    """
        }
    ]

    # Aplicar plantilla de chat
    prompt = ranker_pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    #  Generación
    outputs = ranker_pipeline(
        prompt, 
        max_new_tokens=500,  
        temperature=0.2,     # Temperatura baja para ser analítico
        do_sample=True,
        return_full_text=False
    )
    
    return outputs[0]['generated_text'].strip()


# ==============================================================================
# FUNCIÓN RAG AUTOMATIZADA 
# ==============================================================================
def ejecutar_rag_pipeline(ranker_pipeline, job_offer_query, k=3):
    """Ejecuta el pipeline RAG completo para encontrar y rankear candidatos."""
    
    print(f"\n{'='*60}")
    print(f"INICIANDO RAG para: {job_offer_query}")
    print(f"Buscando los {k} mejores candidatos (usando Cover Letters completas)...")
    print(f"{'='*60}")

    # 1. RECUPERACIÓN (Retrieval - BGE + FAISS)
    candidatos_encontrados = buscar_candidatos(job_offer_query, k=k)

    if not candidatos_encontrados:
        print("\n No se encontraron candidatos relevantes. Terminando el pipeline.")
        return { "ranking_final_llm": "No se encontraron candidatos para rankear." }

    print("\n[FASE 1: RECUPERACIÓN COMPLETADA]")
    for i, c in enumerate(candidatos_encontrados, 1):
        print(f"  {i}. Candidato ID: {c['id']} | Similitud BGE: {c['score']:.4f}")

    # 2. RANKING/GENERACIÓN 
    print(f"\n{'-'*60}")
    print("INICIANDO FASE DE RANKING (LLM)...")
    
    ranking_generado = generar_ranking_llm(ranker_pipeline, job_offer_query, candidatos_encontrados)
    
    print(f"{'-'*60}")
    print("REPORTE FINAL DE RR.HH:")
    print(ranking_generado)

    return { "ranking_final_llm": ranking_generado }


--- 2. Generando Embeddings e Índice FAISS (BGE) ---


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Índice FAISS creado en CPU con 813 vectores.


## Modelo A usando Qwen 3B cuantizado a 4bits

In [6]:
# ==============================================================================
#  CONFIGURACIÓN DEL GENERATOR Qwen
# ==============================================================================
LLM_ID = "Qwen/Qwen2.5-3B"
print(f"\n---  Cargando LLM Generador: {LLM_ID} (Contexto 128K tokens) ---")

# Configuración de 4 bits para ahorrar memoria
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer_A = AutoTokenizer.from_pretrained(LLM_ID)
llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_ID,
    quantization_config=quantization_config,
    device_map="auto"
)

# Configuración del tokenizers para generación causal
tokenizer_A.pad_token = tokenizer_A.eos_token 
tokenizer_A.padding_side = "left" 

ranker_pipeline_A = pipeline(
    "text-generation", 
    model=llm_model,
    tokenizer=tokenizer_A,
    device_map="auto"
)

print("LLM cargado correctamente en memoria reducida (4-bit).")



---  Cargando LLM Generador: Qwen/Qwen2.5-3B (Contexto 128K tokens) ---


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Device set to use cuda:0


LLM cargado correctamente en memoria reducida (4-bit).


## Modelo B usando ... cuantizado a 4bits

In [8]:
# ==============================================================================
#  CONFIGURACIÓN DEL GENERATOR .....
# ==============================================================================
LLM_ID = "microsoft/Phi-3-mini-4k-instruct"
print(f"\n--- Cargando LLM Generador: {LLM_ID} (Contexto 32K tokens) ---")

# Configuración de 4 bits para ahorrar memoria
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer_B = AutoTokenizer.from_pretrained(LLM_ID)
llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_ID,
    quantization_config=quantization_config,
    device_map="auto"
)

# Configuración del tokenizers para generación causal
tokenizer_B.pad_token = tokenizer_B.eos_token 
tokenizer_B.padding_side = "left" 

ranker_pipeline_B = pipeline(
    "text-generation", 
    model=llm_model,
    tokenizer=tokenizer_B,
    device_map="auto"
)

print("LLM cargado correctamente en memoria reducida (4-bit).")



--- Cargando LLM Generador: microsoft/Phi-3-mini-4k-instruct (Contexto 32K tokens) ---


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


LLM cargado correctamente en memoria reducida (4-bit).


## Resultados

In [14]:
TARGET_JOB = "We need a Project Manager with AWS certification, strong leadership, experience in other projects related to finances"


In [15]:
# usando QWEN
ejecutar_rag_pipeline(ranker_pipeline_A, TARGET_JOB, k=3)


INICIANDO RAG para: We need a Project Manager with AWS certification, strong leadership, experience in other projects related to finances
Buscando los 3 mejores candidatos (usando Cover Letters completas)...

[FASE 1: RECUPERACIÓN COMPLETADA]
  1. Candidato ID: 712 | Similitud BGE: 0.6870
  2. Candidato ID: 324 | Similitud BGE: 0.6778
  3. Candidato ID: 129 | Similitud BGE: 0.6638

------------------------------------------------------------
INICIANDO FASE DE RANKING (LLM)...
------------------------------------------------------------
REPORTE FINAL DE RR.HH:
Based on the provided job description and cover letters, Candidate 1 (ID 712) is the best fit for the job. Here's the reasoning:

**BEST CANDIDATE ID:** 712

**OVERALL MATCH:** Best

**REASONING:** Candidate 1 has a strong background in cloud computing, specifically AWS, which aligns perfectly with the job description. Their experience as a Cloud Engineer at VWX Inc and a Network Engineer at STU Corp demonstrates their ability to

{'ranking_final_llm': "Based on the provided job description and cover letters, Candidate 1 (ID 712) is the best fit for the job. Here's the reasoning:\n\n**BEST CANDIDATE ID:** 712\n\n**OVERALL MATCH:** Best\n\n**REASONING:** Candidate 1 has a strong background in cloud computing, specifically AWS, which aligns perfectly with the job description. Their experience as a Cloud Engineer at VWX Inc and a Network Engineer at STU Corp demonstrates their ability to manage and lead projects related to finances. Additionally, their certification in AWS and Azure, proficiency in Java, Python, and SQL, and their passion for leveraging cloud technologies to drive business growth and efficiency further reinforce their suitability for the role. In contrast, Candidate 2 (ID 324) has experience with Java but not specifically in cloud computing, and Candidate 3 (ID 129) has experience with Java and JavaServer, but lacks experience in cloud computing and is not specifically focused on cloud computing pr

In [11]:
# usando ....
ejecutar_rag_pipeline(ranker_pipeline_B, TARGET_JOB, k=3)


INICIANDO RAG para: We need a Project Manager with AWS certification, strong leadership, experience in other projects related to finances
Buscando los 3 mejores candidatos (usando Cover Letters completas)...

[FASE 1: RECUPERACIÓN COMPLETADA]
  1. Candidato ID: 712 | Similitud BGE: 0.6870
  2. Candidato ID: 324 | Similitud BGE: 0.6778
  3. Candidato ID: 129 | Similitud BGE: 0.6638

------------------------------------------------------------
INICIANDO FASE DE RANKING (LLM)...
------------------------------------------------------------
REPORTE FINAL DE RR.HH:
1. **Candidate ID 712**: Good Match
   - **Reasoning**: Candidate ID 712 demonstrates relevant AWS certification and cloud computing experience, which aligns with the job's requirements. The candidate's background in computer engineering and proficiency in cloud technologies, Python, and SQL are pertinent skills for a project manager role. However, the absence of explicit leadership experience and finance-related project experien

{'ranking_final_llm': "1. **Candidate ID 712**: Good Match\n   - **Reasoning**: Candidate ID 712 demonstrates relevant AWS certification and cloud computing experience, which aligns with the job's requirements. The candidate's background in computer engineering and proficiency in cloud technologies, Python, and SQL are pertinent skills for a project manager role. However, the absence of explicit leadership experience and finance-related project experience is noted.\n\n    2. **Candidate ID [ID]**:...\n\n\n### JOB DESCRIPTION:\n    We need a Senior Financial Analyst with at least 10 years of experience, expertise in financial modeling, and a track record of successful project management in finance\n    \n    ### CANDIDATES TO EVALUATE:\n    CANDIDATE 1 (ID 712) - COVER LETTER: Dear Hiring Manager, I am applying for the Cloud Engineer position at PQR Inc. I am a certified AWS and Azure professional with 5 years of experience in cloud computing. I currently work at VWX Inc as a Cloud Engi